## Preprocessing and Feature Extraction

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob 

from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string

In [2]:
df = pd.read_csv("/Users/harveymiller/Documents/GitHub/text-success/Data/Ukraine Tweets.csv")

## Sentiment Analysis:

Steps taken:
- Converting emojis to text
- We decide to remove all the mentions and hashtagged words, as these will be analysed separately
- Remove Links, as these don't contribute to SA
- Conducting the SA on our preprocessed data

Creating a new column so that we can see the adjusted tweet and original versiom

In [3]:
df.insert(loc=6,
          column='Adjusted Tweet',
          value=df['rendered_content'])

Converting emojis to text

In [4]:
import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

Removing mentions and hashtagged words

In [5]:
import re

def remove_mentions_hashtags(text):
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_mentions_hashtags)

Removing links

In [6]:
import re

def remove_links(text):
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_links)

Insert Polarity Score Column

In [7]:
df.insert(loc=7,
          column='Polarity Score',
          value=df['Adjusted Tweet'])

Sentiment Analysis using NLTK's VADER

In [8]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sentiment_analysis(text):  
    text = sia.polarity_scores(text)
    return text

df['Polarity Score'] = df['Polarity Score'].apply(sentiment_analysis)
    

Creating columns for:
- Negative Score
- Neutral Score
- Positive Score
- Compound Score [-1,1]

In [9]:
df.insert(loc=8,
          column='Negative Score',
          value=df['Polarity Score'])

df.insert(loc=9,
          column='Neutral Score',
          value=df['Polarity Score'])

df.insert(loc=10,
          column='Positive Score',
          value=df['Polarity Score'])

df.insert(loc=11,
          column='Compound Score',
          value=df['Polarity Score'])

In [10]:
def negative_score(text):
    text = text['neg']
    return text

df['Negative Score'] = df['Negative Score'].apply(negative_score)

In [11]:
def neutral_score(text):
    text = text['neu']
    return text

df['Neutral Score'] = df['Neutral Score'].apply(neutral_score)

In [12]:
def positive_score(text):
    text = text['pos']
    return text

df['Positive Score'] = df['Positive Score'].apply(positive_score)

In [13]:
def compound_score(text):
    text = text['compound']
    return text

df['Compound Score'] = df['Compound Score'].apply(compound_score)

Sentiment Analysis using TextBlob

In [14]:
df.insert(loc=12,
          column='Polarity Score_textblob',
          value=df['Adjusted Tweet'])

In [15]:
df.insert(loc=13,
          column='Subjectivity Score_textblob',
          value=df['Adjusted Tweet'])

In [16]:
from textblob import TextBlob

#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Polarity Score_textblob'] = df['Polarity Score_textblob'].apply(getPolarity)
df['Subjectivity Score_textblob'] = df['Subjectivity Score_textblob'].apply(getSubjectivity)


## Further manipulating the tweet

Steps taken:
- Lowercase
- Punctuation
- Tokenization
- Stopword filtering
- Stemming

Changing all text to lowercase

In [17]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

Removing all Punctuation

In [18]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

Tokenizing

In [19]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

Stopword Filtering

In [20]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_stopwords)

Stemming

In [21]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

In [31]:
df.head()

,id,date,user,user_followers,user_created,rendered_content,Adjusted Tweet,Copy,Polarity Score,Negative Score,...,Polarity Score_textblob,Subjectivity Score_textblob,likes,retweets,replies,quoteCount,hashtags,lang,media,mentionedUsers
0,1477420789863436289,2022-01-01 23:25:40+00:00,anno1540,8838,2014-06-12 17:05:22+00:00,"Lithuania will never abandon Ukraine, voluntee...","[lithuania, never, abandon, ukraine, volunteer...",,"{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'comp...",0.000,...,0.000000,0.000000,5,1,0,0,"['Lithuania', 'Ukraine']",en,NaN,NaN
1,1477414596424220679,2022-01-01 23:01:03+00:00,weather_odessa,119,2019-07-10 08:34:22+00:00,#odessa #odesa #ukraine #одесса\nNow: 4.2°C\nT...,"[одесса, 42°c, todays, min, 42°c, 002852, toda...",,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,...,0.000000,0.000000,0,0,0,0,"['odessa', 'odesa', 'ukraine', 'одесса']",en,NaN,NaN
2,1477414332376010752,2022-01-01 23:00:00+00:00,AlArabiya_Eng,927174,2009-02-28 08:31:32+00:00,After tough talk between Presidents Joe Biden ...,"[tough, talk, presidents, joe, biden, vladimir...",biden,"{'neg': 0.099, 'neu': 0.776, 'pos': 0.125, 'co...",0.099,...,-0.194444,0.666667,4,0,3,0,"['Russia', 'Ukraine']",en,NaN,NaN
3,1477409748572151809,2022-01-01 22:41:47+00:00,beatravelling,6329,2014-02-28 21:25:33+00:00,The beach can be nice in the fall too 😊🇺🇦\n\n#...,"[beach, nice, fall, smilingfacewithsmilingeyes...",,"{'neg': 0.0, 'neu': 0.781, 'pos': 0.219, 'comp...",0.000,...,0.600000,1.000000,0,0,0,0,"['lanzheron', 'langeron', 'beach', 'odessa', '...",en,NaN,NaN
4,1477409332820119552,2022-01-01 22:40:08+00:00,TornCurtain1991,677,2012-02-08 15:30:41+00:00,"A note: Stepan #Bandera, DOB 01011909, was lea...","[note, stepan, dob, 01011909, leader, organiza...",,"{'neg': 0.171, 'neu': 0.829, 'pos': 0.0, 'comp...",0.171,...,-0.100000,0.033333,1,2,0,0,"['Bandera', 'Ukraine']",en,NaN,NaN


## One Hot encoding for specific words of our choice

In [23]:
df.insert(loc=7,
          column='Copy',
          value= df['Adjusted Tweet'])

In [26]:
def count(text):
    bank = ['zelensky','zelenskiy','biden','johnson']
    text = [word for word in text if word in bank]
    return text

df['Copy'] = df['Copy'].apply(count)

In [28]:
def replicate_remove(text):
    return list(dict.fromkeys(text))

df['Copy'] = df['Copy'].apply(replicate_remove)

In [30]:
def list_to_string(text):
    text = "".join(str(char) for char in text)
    return text

df['Copy'] = df['Copy'].apply(list_to_string)

In [33]:
pd.get_dummies(df['Copy'], prefix='Count')

,Count_,Count_biden,Count_bidenjohnson,Count_bidenzelenskiy,Count_bidenzelensky,Count_johnson,Count_johnsonbiden,Count_johnsonzelenskiy,Count_johnsonzelensky,Count_zelenskiy,Count_zelenskiybiden,Count_zelensky,Count_zelenskybiden,Count_zelenskyjohnson
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60128,1,0,0,0,0,0,0,0,0,0,0,0,0,0
60129,1,0,0,0,0,0,0,0,0,0,0,0,0,0
60130,0,1,0,0,0,0,0,0,0,0,0,0,0,0
60131,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
bank = ['zelensky','zelenskiy','biden','johnson']

def column_insert(i):
    df.insert(loc=7,
          column= i+' Count',
          value= df['Copy'])

for i in bank:
    column_insert(i)

In [ ]:
bank = ['zelensky','zelenskiy','biden','johnson']

for c in bank:
    def one_hot_encode(text):
        if str(text) == c:
            return 1
        else:
            return 0
    
    df[c + ' Count'] = df[c + ' Count'].apply(one_hot_encode)

## Months since creation of account relative to tweet

In [ ]:
#Finding date of account creation in months
df.insert(loc=5,
          column='Date of Creation in months',
          value=df['user_created'])

In [ ]:
from datetime import *

#returning the months of account creation
def account_creation(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    year = str(text)[0:4]
    month = str(text)[5:7]
    total_months = (int(year)*12)+(int(month))
    return (total_months)

df['Date of Creation in months'] = df['Date of Creation in months'].apply(account_creation)

In [ ]:
#Finding date of tweet in months
df.insert(loc=6,
          column='Date of Tweet in Months',
          value=df['date'])

In [ ]:
from datetime import *

#return the year of tweet
df['Date of Tweet in Months'] = df['Date of Tweet in Months'].apply(account_creation)

In [ ]:
#Calculating Months since creation of account relative to tweet
df.insert(loc=7,
          column='Months Since Creation of Account',
          value= (df['Date of Tweet in Months']-df['Date of Creation in months']))

## Time of Day

In [ ]:
#create a column for hours:
df.insert(loc=2,
          column='hour of tweet',
          value=df['date'])

In [ ]:
from datetime import *

#return the hour of the tweet
def hour(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    hour = str(text.time())[0:2]
    return int(hour)

df['hour of tweet'] = df['hour of tweet'].apply(hour)

In [ ]:
#insert column for the time of day
df.insert(loc=3,
          column='time of day',
          value=df['hour of tweet'])

In [ ]:
#calculating the time of day
def time_of_day(text):  
    if ((text > 4) and (text < 8 )):
        return 'Early Morning'
    elif ((text > 8) and (text < 12 )):
        return 'Morning'
    elif ((text > 12) and (text < 16 )):
        return 'Noon'
    elif ((text > 16) and (text < 20 )):
        return 'Eve'
    elif ((text > 20) and (text < 24 )):
        return 'Night'
    elif ((text > 0) and (text < 4 )):
        return 'Late Night'
    
df['time of day'] = df['time of day'].apply(time_of_day)


In [ ]:
#Creating counts using one hot encoding

#Early Morning Count
df.insert(loc=4,
          column='Early Morning Count',
          value=df['time of day'])

def early_morning_count(text):
    if text == 'Early Morning':
        return 1
    else:
        return 0

df['Early Morning Count'] = df['Early Morning Count'].apply(early_morning_count)

#Morning Count
df.insert(loc=5,
          column='Morning Count',
          value=df['time of day'])

def morning_count(text):
    if text == 'Morning':
        return 1
    else:
        return 0

df['Morning Count'] = df['Morning Count'].apply(morning_count)

#Noon count
df.insert(loc=6,
          column='Noon Count',
          value=df['time of day'])

def noon_count(text):
    if text == 'Noon':
        return 1
    else:
        return 0

df['Noon Count'] = df['Noon Count'].apply(noon_count)

#Eve count
df.insert(loc=7,
          column='Eve Count',
          value=df['time of day'])

def eve_count(text):
    if text == 'Eve':
        return 1
    else:
        return 0

df['Eve Count'] = df['Eve Count'].apply(eve_count)

#Night count
df.insert(loc=8,
          column='Night Count',
          value=df['time of day'])

def night_count(text):
    if text == 'Night':
        return 1
    else:
        return 0

df['Night Count'] = df['Night Count'].apply(night_count)

#Late Night count
df.insert(loc=9,
          column='Late Night Count',
          value=df['time of day'])

def late_night_count(text):
    if text == 'Late Night':
        return 1
    else:
        return 0

df['Late Night Count'] = df['Late Night Count'].apply(late_night_count)

## Video, GIF and Photo Count

Photo Count

In [ ]:
#Creating a photo count column...
df.insert(loc=23,
          column='Photo Count',
          value=df['media'])

In [ ]:
#Counting number of Photos in media column
#No need to tokenize
def photo_count(text):
    text = str(text)
    text = text.count('Photo')
    return text

df['Photo Count'] = df['Photo Count'].apply(photo_count)


Video Count

In [ ]:
#Creating a video count column...
df.insert(loc=24,
          column='Video Count',
          value=df['media'])

In [ ]:
#We need to tokenize the media column so that we can count how many videos there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['Video Count'] = df['Video Count'].apply(tokenize)

In [ ]:
#Counting number of Videos in media column
def video_count(text):
    text = text.count('Video')
    return text

df['Video Count'] = df['Video Count'].apply(video_count)

Gif Count

In [ ]:
#Creating a GIF count column...
df.insert(loc=25,
          column='GIF Count',
          value=df['media'])

In [ ]:
#We need to tokenize the media column so that we can count how many GIFs there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['GIF Count'] = df['GIF Count'].apply(tokenize)

In [ ]:
#Counting number of GIFs in media column
def gif_count(text):
    text = text.count('Gif')
    return text

df['GIF Count'] = df['GIF Count'].apply(gif_count)

## Creating our final Dataframe

Steps taken:
- Drop: media, date, user, polarity score, user_created, Date of Creation in months, Date of Tweet in Months, rendered_content.	

In [ ]:
df.drop(['media','date','user','Polarity Score','user_created','Date of Creation in months','Date of Tweet in Months','rendered_content'], axis=1)